# Apify | Apify

このノートブックは、LangChainのための[Apify integration](/docs/ecosystem/integrations/apify)の使用方法を示しています。

> This notebook shows how to use the [Apify integration](/docs/ecosystem/integrations/apify) for LangChain.

[Apify](https://apify.com)は、ウェブスクレイピングやデータ抽出のためのクラウドプラットフォームで、様々なウェブスクレイピング、クローリング、データ抽出のユースケースに対応する、*Actors*と呼ばれる千を超える既製アプリの[エコシステム](https://apify.com/store)を提供しています。例えば、Google検索結果、InstagramやFacebookのプロフィール、AmazonやShopifyの商品、Googleマップのレビューなどを抽出するために使用できます。

> [Apify](https://apify.com) is a cloud platform for web scraping and data extraction,
> which provides an [ecosystem](https://apify.com/store) of more than a thousand
> ready-made apps called *Actors* for various web scraping, crawling, and data extraction use cases.
> For example, you can use it to extract Google Search results, Instagram and Facebook profiles, products from Amazon or Shopify, Google Maps reviews, etc. etc.

この例では、ドキュメント、ナレッジベース、ヘルプセンター、ブログなどのウェブサイトを深くクロールし、ウェブページからテキストコンテンツを抽出する [Website Content Crawler](https://apify.com/apify/website-content-crawler) Actorを使用します。その後、ドキュメントをベクターインデックスに取り込んで、それを使って質問に答えます。

> In this example, we'll use the [Website Content Crawler](https://apify.com/apify/website-content-crawler) Actor,
> which can deeply crawl websites such as documentation, knowledge bases, help centers, or blogs,
> and extract text content from the web pages. Then we feed the documents into a vector index and answer questions from it.



In [ ]:
#!pip install apify-client openai langchain chromadb tiktoken

まず、`ApifyWrapper`をソースコードにインポートしてください：

> First, import `ApifyWrapper` into your source code:



In [1]:
from langchain.document_loaders.base import Document
from langchain.indexes import VectorstoreIndexCreator
from langchain.utilities import ApifyWrapper

以下のリンクから取得できる[Apify APIトークン](https://console.apify.com/account/integrations)を使用して初期化し、この例のためにはあなたのOpenAI APIキーも併せて使用してください：

> Initialize it using your [Apify API token](https://console.apify.com/account/integrations) and for the purpose of this example, also with your OpenAI API key:



In [2]:
import os

os.environ["OPENAI_API_KEY"] = "Your OpenAI API key"
os.environ["APIFY_API_TOKEN"] = "Your Apify API token"

apify = ApifyWrapper()

その後、Actorを実行して完了するまで待ち、ApifyデータセットからLangChainドキュメントローダーにその結果を取得してください。

> Then run the Actor, wait for it to finish, and fetch its results from the Apify dataset into a LangChain document loader.

Apifyのデータセットにすでにいくつかの結果がある場合、[このノートブック](/docs/integrations/document_loaders/apify_dataset)に示されているように`ApifyDatasetLoader`を使用して直接ロードすることができます。そのノートブックでは、ApifyデータセットのレコードからLangChainの`Document`フィールドへフィールドをマッピングする`dataset_mapping_function`の説明も含まれています。

> Note that if you already have some results in an Apify dataset, you can load them directly using `ApifyDatasetLoader`, as shown in [this notebook](/docs/integrations/document_loaders/apify_dataset). In that notebook, you'll also find the explanation of the `dataset_mapping_function`, which is used to map fields from the Apify dataset records to LangChain `Document` fields.



In [3]:
loader = apify.call_actor(
    actor_id="apify/website-content-crawler",
    run_input={"startUrls": [{"url": "https://python.langchain.com/en/latest/"}]},
    dataset_mapping_function=lambda item: Document(
        page_content=item["text"] or "", metadata={"source": item["url"]}
    ),
)

クロールされたドキュメントからベクターインデックスを初期化します：

> Initialize the vector index from the crawled documents:



In [ ]:
index = VectorstoreIndexCreator().from_loaders([loader])

最後に、ベクターインデックスを照会します：

> And finally, query the vector index:



In [5]:
query = "What is LangChain?"
result = index.query_with_sources(query)

In [6]:
print(result["answer"])
print(result["sources"])

 LangChain is a standard interface through which you can interact with a variety of large language models (LLMs). It provides modules that can be used to build language model applications, and it also provides chains and agents with memory capabilities.

https://python.langchain.com/en/latest/modules/models/llms.html, https://python.langchain.com/en/latest/getting_started/getting_started.html
